### Amazon product details scraped 

 - URL
 - Name
 - Price
 - Review Count
 - Rating 
 

In [1]:
# import required packeges for webscrape

import csv
from bs4 import BeautifulSoup
from selenium import webdriver 



In [2]:
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/unachka/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


In [3]:
def get_url (search_term):
    """Generate a url from search text"""
    template ='https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
    search_term = search_term.replace(' ', '+')
    
    #add term query to url
    url = template.format(search_term)
    
    #add page query placeholder
    url+='&page={}'
    return url

def extract_record(item):
    """Extract and return dta from single record"""
    
    #description and url
    atag = item.h2.a
    description=atag.text.strip()
    url = 'https//www.amazon.com' + atag.get('href')
    
    try:
        #price
        #if price is empty i don't the item in my list
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try:
        #rank and rating
        #if the review I still want to return the the product so, returning empty cell
        rating = item.i.text[0:3]
        review_count = item.find('span',{'class': 'a-size-base', 'dir': 'auto'}).text
    except AttributeError:
        rating =''
        review_count =''
        
    result = (description, price, rating, review_count, url )
    
    return result

def main(search_term):
    """Run main program routine"""
    
    #startup the webdriver
    driver = webdriver.Chrome(ChromeDriverManager().install())
    
    records =[]
    url = get_url(search_term)
    
    for page in range (1, 21): #iterating through pages 
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
    driver.close() #close the application 
    
      
    with open('results_dark_spot.csv', 'w', newline='', encoding='utf-8') as f:
        
        """Save the result as csv file"""    
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'Url'])
        writer.writerows(records)


In [4]:
main('dark spot remover for face')

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/unachka/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache
